<a href="https://colab.research.google.com/github/helenabarmer/federated-learning-on-raspberry-pi/blob/master/helenabarmer/FederatedLearningRaspberryPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Install PySyft in Google Colab

!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.1MB 4.2MB/s 
     |████████████████████████████████| 276kB 41.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 28749, done.
remote: Total 28749 (delta 0), reused 0 (delta 0), pack-reused 28749
Receiving objects: 100% (28749/28749), 32.08 MiB | 19.65 MiB/s, done.
Resolving deltas: 100% (19066/19066), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 4.9MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.1.21a1 has requirement tf_encrypted!=0.5.7,>=0.5.4, b

In [8]:
from __future__ import unicode_literals, print_function, division
from torch.utils.data import Dataset

import torch
from io import open
import glob
import os
import numpy as np
import unicodedata
import string
import random
import torch.nn as nn
import time
import math
import syft as sy
import pandas as pd
import random
from syft.frameworks.torch.federated import utils

from syft.workers import WebsocketClientWorker
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

W0724 08:58:31.090229 140131523962752 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0724 08:58:31.110163 140131523962752 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
!wget https://download.pytorch.org/tutorial/data.zip  

--2019-07-24 08:55:52--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.224.29.73, 13.224.29.19, 13.224.29.48, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.224.29.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.08s   

2019-07-24 08:55:52 (35.5 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating: data/names/Vietnamese.txt  


In [9]:
path = '/content/data/names/*.txt'

all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)


#Load files in the path
def findFiles(path):
  return glob.glob(path)

#Read a file and then split to lines
def readLines(filename):
  lines =open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicodeToAscii(line) for line in lines]

#Convert  string to ASCII format
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
      and c in all_letters
  )



category_lines = {}
all_categories = []

for filename in findFiles(path):
  #print(filename)
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readLines(filename)
  category_lines[category] = lines
  
n_categories = len(all_categories)

#for names in glob.glob(path):
  #print(names)
  
 
print("Number of categories: " + "\n" + str(n_categories))
print("\n" + "All categories: ")
print(*all_categories, sep = "\n")

Number of categories: 
18

All categories: 
Japanese
Dutch
Arabic
Chinese
Russian
German
Vietnamese
French
Scottish
Korean
Italian
Czech
Polish
English
Portuguese
Spanish
Irish
Greek


In [10]:
print(*category_lines['Polish'][:6], sep = "\n")

Adamczak
Adamczyk
Andrysiak
Auttenberg
Bartosz
Bernard
